In [ ]:
#r "B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
//#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
var myBatch = (SlurmClient)GetDefaultQueue();
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+2000});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands

index,value
0,#SBATCH -C avx512
1,#SBATCH --mem-per-cpu=2000


In [ ]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; myDB

Project name is set to 'Benchmark_dbname'.
Opening existing database 'W:\work\scratch\jw52xeqa\Benchmark_dbname'.


{ Session Count = 18; Grid Count = 11; Path = W:\work\scratch\jw52xeqa\Benchmark_dbname }

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

In [ ]:

int[] Resolutions_3D = new int[] { 4 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    //double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    //double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    //double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    string prjname = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.CurrentProject;
    string GridName = string.Format(prjname + "_J" + J);
    
    try{
        grids[cnt]=null;
        foreach(IGridInfo grd in myDB.Grids){
            bool check = grd.Name.Contains(string.Format("_J"+ J));
            if(check){
            grids[cnt]=grd;
            }
        }
    } catch (Exception ex){
        //Console.WriteLine(ex.Message);
    }
    
    //grids[cnt] = tempDB.Grids.SingleOrDefault(grd => grd.Name == GridName);
    
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            if (Math.Abs(X[0] - (-1)) <= 1.0e-8)
                return "wall_left";
            if (Math.Abs(X[0] - (+1)) <= 1.0e-8)
                return "wall_right";
            if (Math.Abs(X[1] - (-1)) <= 1.0e-8)
                return "wall_front";
            if (Math.Abs(X[1] - (+1)) <= 1.0e-8)
                return "wall_back";
            if (Math.Abs(X[2] - (-1)) <= 1.0e-8)
                return "wall_top";
            if (Math.Abs(X[2] - (+1)) <= 1.0e-8)
                return "wall_bottom";
            throw new ArgumentException("unknown wall");
        });
      
        myDB.SaveGrid(ref g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Object reference not set to an instance of an object.
Found Grid: { Guid = f5c62319-0ca0-4544-8ac3-a0c1be2a012e; Name = Benchmark_dbname_J64; Cell Count = 64; Dim = 3 }


In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

In [ ]:
Func<LinearSolverCode, int, IGridInfo, int ,XNSE_Control> GenerateCObj = delegate (LinearSolverCode solver, int p, IGridInfo grid, int DOF){
    
    // ProjectName
    // ============
    int NoSblocks = 4;
    int J         = grid.NumberOfCells;
    
    string caseName = string.Format("J{0}_p{1}_{2}", J, p, solver);
    Console.WriteLine("setting up: " + caseName);
    
    XNSE_Control C       = new XNSE_Control();
    C.SetDatabase(myDB);
    C.savetodb           = true;
    C.ProjectName        = WFlowName;
    C.ProjectDescription = "static droplet";
    C.SessionName        = caseName;
    C.GridGuid           = grid.ID;
    
    // DG degrees
    // ============
    
    C.FieldOptions.Add("Velocity*", new FieldOpts() {
        Degree = p,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Pressure", new FieldOpts() {
        Degree = p - 1,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });    
    C.FieldOptions.Add("GravityY", new FieldOpts() {
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("PhiDG", new FieldOpts() {
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Phi", new FieldOpts() {
        Degree = 2,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Curvature", new FieldOpts() {
        Degree = Math.Max(4, 2*p + 2),
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    
    // Phys. Parameters
    // ============
    
    C.PhysicalParameters.rho_A             = 1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    C.TracingNamespaces                    = "*";
    
    // Dont know
    // ============
    
    double r     = 0.49;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2()-1", false));
    
    C.LSContiProjectionMethod = ContinuityProjectionOption.None;
    C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    //C.AdvancedDiscretizationOptions.CellAgglomerationThreshold = 0.2;
    //C.AdvancedDiscretizationOptions.PenaltySafety = 40;
    //C.AdvancedDiscretizationOptions.UseGhostPenalties = true;
    //C.ContiField = XNSE_Control.ContinuityProjection.ContinuousDG;
    
    // Solver Stuff
    // ============
    
    //C.VelocityBlockPrecondMode         = MultigridOperator.Mode.IdMass_DropIndefinite;
    //C.PressureBlockPrecondMode         = MultigridOperator.Mode.IdMass_DropIndefinite;
    C.UseSchurBlockPrec                = true;
    C.LinearSolver.NoOfMultigridLevels = 5;
    C.LinearSolver.MaxSolverIterations = 5000;
    C.LinearSolver.pMaxOfCoarseSolver  = p;
    
    C.LinearSolver.TargetBlockSize = Math.Min(J*DOF/2,10000); //Make at least two blocks with Schwarz plz...
    //C.LinearSolver.TargetBlockSize = (int)(J*DOF/(NoSblocks-1));

    C.LinearSolver.ConvergenceCriterion = 1e-8;
    C.LevelSet_ConvergenceCriterion     = 1e-6;
    C.LinearSolver.SolverCode           = solver;
    C.LinearSolver.verbose              = true;
    C.NonLinearSolver.verbose           = true;

    C.Option_LevelSetEvolution                          = LevelSetEvolution.None;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.FullySymmetric;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    // Timestepping
    // ============

    C.TimeSteppingScheme           = TimeSteppingScheme.ImplicitEuler;
    C.Timestepper_BDFinit          = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    //C.TimesteppingMode             = AppControl._TimesteppingMode.Transient;
    //C.dtFixed                      = 1E-4;
            
    C.SessionName += "_pLow"+C.LinearSolver.pMaxOfCoarseSolver ;
    return C;
    };

In [ ]:
Func<LinearSolverCode, int, IGridInfo, int ,XNSE_Control> GenerateChannel = delegate (LinearSolverCode solver, int p, IGridInfo grid, int DOF){
    
    // ProjectName
    // ============
    int J         = grid.NumberOfCells;
    
    string caseName = string.Format("J{0}_p{1}_{2}", J, p, solver);
    Console.WriteLine("setting up: " + caseName);
    
    XNSE_Control C       = new XNSE_Control();
    C.SetDatabase(myDB);
    C.savetodb           = true;
    C.ProjectName        = WFlowName;
    C.ProjectDescription = "static droplet";
    C.SessionName        = caseName;
    C.GridGuid           = grid.ID;
    
    // DG degrees
    // ============
    
    C.FieldOptions.Add("Velocity*", new FieldOpts() {
        Degree = p,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Pressure", new FieldOpts() {
        Degree = p - 1,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });    
    C.FieldOptions.Add("GravityY", new FieldOpts() {
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    /*
    C.FieldOptions.Add("PhiDG", new FieldOpts() {
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Phi", new FieldOpts() {
        Degree = 2,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Curvature", new FieldOpts() {
        Degree = Math.Max(4, 2*p + 2),
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    */
    
    // Phys. Parameters
    // ============
    
    C.PhysicalParameters.rho_A             = 1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    C.TracingNamespaces                    = "*";
    
    C.LSContiProjectionMethod = ContinuityProjectionOption.None;
    C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;
    
    // Solver Stuff
    // ============
    
    C.UseSchurBlockPrec                = true;
    C.LinearSolver.NoOfMultigridLevels = 5;
    C.LinearSolver.MaxSolverIterations = 5000;
    C.LinearSolver.pMaxOfCoarseSolver  = p;
    
    C.LinearSolver.TargetBlockSize = Math.Min(J*DOF/2,10000); //Make at least two blocks with Schwarz plz...
    //C.LinearSolver.TargetBlockSize = (int)(J*DOF/(NoSblocks-1));

    C.LinearSolver.ConvergenceCriterion = 1e-8;
    C.LinearSolver.SolverCode           = solver;
    C.LinearSolver.verbose              = true;
    C.NonLinearSolver.verbose           = true;

    C.Option_LevelSetEvolution                          = LevelSetEvolution.None;
    C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.FullySymmetric;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    double VelocityIn = 1.0;
    C.AddBoundaryValue("Velocity_inlet", "VelocityX", new Formula($"(X,t) => {VelocityIn}", false));
    C.AddInitialValue("VelocityX", new Formula($"(X) => {VelocityIn}"));

    // Timestepping
    // ============

    C.TimeSteppingScheme           = TimeSteppingScheme.ImplicitEuler;
    C.Timestepper_BDFinit          = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    //C.TimesteppingMode             = AppControl._TimesteppingMode.Transient;
    C.dtFixed                      = double.MaxValue;
            
    C.SessionName += "_pLow"+C.LinearSolver.pMaxOfCoarseSolver ;
    return C;
    };

In [ ]:
int[] PolyDegS = new int[] {3};
//int[] PolyDegS = new int[] {2};

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    //LinearSolverCode.classic_pardiso,
    //LinearSolverCode.exp_OrthoS_pMG,
    //LinearSolverCode.exp_gmres_levelpmg,
    //LinearSolverCode.exp_softgmres_schwarz_directcoarse_overlap,
    LinearSolverCode.exp_Kcycle_schwarz,
    //LinearSolverConfig.Code.classic_mumps, 
    //LinearSolverConfig.Code.classic_cg,
    //LinearSolverConfig.Code.exp_softpcg_schwarz_directcoarse,
    //LinearSolverConfig.Code.exp_softpcg_mg
}; 

In [ ]:
//List<IGridInfo> t_grids=new List<IGridInfo>();
//grids.Pick(1,2,3,4,5,6,7).ForEach(g => t_grids.Add(g));
//t_grids.ToArray();
IGridInfo[] t_grids = grids;
t_grids

#0: { Guid = f5c62319-0ca0-4544-8ac3-a0c1be2a012e; Name = Benchmark_dbname_J64; Cell Count = 64; Dim = 3 }


In [ ]:
// MaxDof for one claculation (we skip fine grids for higher polynomial orders)
int MaxDof = 5000000;

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in t_grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3+Np_p;
    if(J*DOF > MaxDof)
        continue;
    
    var Ctrl = GenerateCObj(solver_name,k,grd,DOF);
    
    controls.Add(Ctrl);
}
}
}

setting up: J64_p3_exp_Kcycle_schwarz


In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(XNSE));
    aJob.SetControlObject(ctrl);
    aJob.NumberOfMPIProcs         = 1;
    aJob.ExecutionTime            = "1:00:00";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
}

 Submitting: J64_p3_exp_Kcycle_schwarz_pLow3
Deploying job J64_p3_exp_Kcycle_schwarz_pLow3 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\Benchmark_dbname-XNSE_Solver2022Feb03_152752
copied 49 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
stderr received: rm: cannot remove 'core.*': No such file or directory
rm: cannot remove 'mono_crash.*': No such file or directory
rm: cannot remove 'mono_crash.mem.*': No such file or directory
Lmod: loading git 2.31.0 
Lmod: loading gcc 9.2.0 
Lmod: loading cmake 3.19.3 
Lmod: loading mono 6.0.0 
Lmod: loading openblas 0.2.20 
Lmod: loading openucx 1.11.2 
Lmod: loading openmpi 4.0.5 
Lmod: loading matlab R2021b 
----- (end of stderr)
stderr received: rm: cannot remove 'core.*': No such file or directory
rm: cannot remove 'mono_crash.*': No such file or directory
rm: cannot remove 'mono_crash.mem.*': No such file or directory
Lmod: loading git 2.31.0 
Lmod: loading gcc 9.